In [1]:
!pip install fosforml

     |████████████████████████████████| 1.9MB 34.3MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 85.2MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 74.8MB/s eta 0:00:01
     |████████████████████████████████| 98.2MB 253kB/s  eta 0:00:012MB/s eta 0:00:01
     |████████████████████████████████| 61kB 10.8MB/s eta 0:00:01
     |████████████████████████████████| 13.1MB 76.0MB/s eta 0:00:01
     |████████████████████████████████| 81kB 18.8MB/s eta 0:00:01
     |████████████████████████████████| 18.3MB 75.5MB/s eta 0:00:01
     |████████████████████████████████| 39.9MB 82.9MB/s eta 0:00:01
     |████████████████████████████████| 174kB 94.7MB/s eta 0:00:01
     |████████████████████████████████| 51kB 12.4MB/s eta 0:00:01
     |████████████████████████████████| 38.6MB 73.7MB/s eta 0:00:01
     |████████████████████████████████| 133kB 113.8MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 113kB/s s eta 0:00:01      | 76.7MB 75.5MB/s eta 0:00:02
   

In [1]:
from fosforml.model_manager.snowflakesession import get_session
session = get_session()

In [2]:
from fosforml.model_manager import snowflakesession
session_instance = snowflakesession()
params = session_instance.connection_params
print(params)

{'user': 'SUNNYCHUGH', 'password': 'Password@1209', 'account': 'rwb76633', 'role': 'SUNNYCHUGH', 'warehouse': 'FOSFOR_WAREHOUSE', 'database': 'INSIGHT_DESIGNER_SPCS', 'schema': 'INSIGHT_SPCS_SCHEMA'}


#### Snowflake Modeling

In [3]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import GradientBoostingClassifier
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake

In [4]:
def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [6]:
# snow_df = session.table("HR_CHURN")
import pandas as pd
df =pd.read_csv("HR_CHURN.csv")
snow_df = session.create_dataframe(df)
input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])

In [7]:
input_data_frame = input_data_frame.drop("Unnamed: 0")
test_df = test_df.drop("Unnamed: 0")

In [8]:
label_encoder_column = []     
input_data_frame = apply_label_encoding(input_data_frame)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']

In [9]:
input_data_frame.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |7.0           |0.38                  |0.53               |2                 |157                     |3                     |0                |1       |0                        |
|2.0       |7.0           |0.11                  |0.88               |7                 |272                     |4                     |0                |1       |0                        |
|1.0       |7.0           |0.37              

In [10]:
pipeline = GradientBoostingClassifier(input_cols=feature_cols, label_cols=target_col)
pipeline.fit(input_data_frame)

The version of package 'snowflake-snowpark-python' in the local environment is 1.20.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


In [10]:
pipeline.label_cols

['SALARY']

In [11]:
!pip freeze | grep -i sci

scikit-learn==1.3.2
scipy==1.13.1


In [11]:
test_df = apply_label_encoding(test_df)

In [12]:
pred_df = pipeline.predict(test_df)

The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


In [13]:
pred_df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"OUTPUT_SALARY"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2.0       |7.0           |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |1.0              |
|1.0       |7.0           |0.72                  |0.87               |5                 |223                     |5                     |0                |1       |

#### Model Registration,

In [14]:
from fosforml import register_model

In [15]:
register_model(
  model_obj=pipeline,
  session=session,
  name="SAURABH_HR_CHURN_22July",
  snowflake_df=pred_df,
  dataset_name="HR_CHURN",
  dataset_source="SnowflakeDataset",
  description="This is a test model",
  flavour="snowflake",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

Calculating build time metrics

Progress: ██████████████                                                         20.0%
Calculating build time metrics

Progress: ████████████████████████████                                           40.0%


The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.
DataFrame.flatten() is deprecated since 0.7.0. Use `DataFrame.join_table_function()` instead.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.


Calculating build time metrics

Progress: ██████████████████████████████████████████                             60.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████████               80.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
Failed to set tags for model MODEL_E237AE1B_F676_46E0_A829_0EE7B6185891_FDC_SAURABH_HR_CHURN_22JULY.
(1304): 01b5d505-0002-a40c-0033-ac8700ace166: 000002 (0A000): Unsupported feature 'TAG'.


"Model 'MODEL_E237AE1B_F676_46E0_A829_0EE7B6185891_FDC_SAURABH_HR_CHURN_22JULY' registered successfully."

In [16]:
session.sql("SELECT TO_CHAR(CURRENT_TIMESTAMP,'YYYY-MM-DD HH12:MI:SS PM')").to_pandas().squeeze()

'2024-07-16 07:52:42 AM'

In [17]:
current_data = pipeline.predict_proba(test_df)
predict_col = list(pipeline.predict(test_df).to_pandas()['OUTPUT_SALARY'].to_numpy())
from snowflake.snowpark.functions import lit
current = None
for i in range(len(predict_col)):
    current = current_data.with_column('OUTPUT_SALARY',lit(predict_col[i]))

In [18]:
current_pandas = current.to_pandas()
current_pandas.head()

,SALARY,DEPARTMENT,SATISFACTION_LEVEL,LAST_EVALUATION,NUMBER_PROJECT,AVERAGE_MONTLY_HOURS,TIME_SPEND_COMPANY,WORK_ACCIDENT,LEFT,PROMOTION_LAST_5YEARS,"""PREDICT_PROBA_0.0""","""PREDICT_PROBA_1.0""","""PREDICT_PROBA_2.0""",OUTPUT_SALARY
0,1.0,2.0,0.38,0.50,2,132,3,0,1,0,0.027671,0.535198,0.437131,1.0
1,2.0,6.0,0.90,0.98,4,264,6,0,1,0,0.018951,0.644719,0.336330,1.0
2,1.0,7.0,0.11,0.97,6,284,4,0,1,0,0.018491,0.643398,0.338111,1.0
3,2.0,9.0,0.37,0.48,2,137,3,0,1,0,0.031146,0.551970,0.416884,1.0
4,1.0,5.0,0.40,0.52,2,155,3,0,1,0,0.038289,0.589165,0.372546,1.0


In [19]:
reference_data = pipeline.predict_proba(input_data_frame)
predict_col = list(pipeline.predict(input_data_frame).to_pandas()['OUTPUT_SALARY'].to_numpy())
from snowflake.snowpark.functions import lit
reference = None
for i in range(len(predict_col)):
    reference = reference_data.with_column('OUTPUT_SALARY',lit(predict_col[i]))

In [20]:
reference_pandas = reference.to_pandas()
reference_pandas.head()

,SALARY,DEPARTMENT,SATISFACTION_LEVEL,LAST_EVALUATION,NUMBER_PROJECT,AVERAGE_MONTLY_HOURS,TIME_SPEND_COMPANY,WORK_ACCIDENT,LEFT,PROMOTION_LAST_5YEARS,"""PREDICT_PROBA_0.0""","""PREDICT_PROBA_1.0""","""PREDICT_PROBA_2.0""",OUTPUT_SALARY
0,1.0,7.0,0.37,0.52,2,159,3,0,1,0,0.027898,0.658963,0.313138,1.0
1,1.0,7.0,0.84,0.92,4,234,5,0,1,0,0.020352,0.659914,0.319734,1.0
2,1.0,7.0,0.82,0.87,4,239,5,0,1,0,0.025122,0.662690,0.312188,1.0
3,1.0,9.0,0.41,0.54,2,152,3,0,1,0,0.025384,0.665482,0.309134,1.0
4,1.0,8.0,0.40,0.54,2,148,3,0,1,0,0.024487,0.697544,0.277969,1.0


In [21]:
inference_pandas_data = test_df.to_pandas()

In [22]:
session.write_pandas(reference_pandas,table_name="HR_REFERENCE_DATA",overwrite=True).show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"""PREDICT_PROBA_0.0"""  |"""PREDICT_PROBA_1.0"""  |"""PREDICT_PROBA_2.0"""  |"OUTPUT_SALARY"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |7.0           |0.37                  |0.52               |2                 |159                     |3                     |0 

In [23]:
session.write_pandas(current_pandas,table_name="HR_CURRENT_DATA",overwrite=True).show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"""PREDICT_PROBA_0.0"""  |"""PREDICT_PROBA_1.0"""  |"""PREDICT_PROBA_2.0"""  |"OUTPUT_SALARY"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |2.0           |0.38                  |0.5                |2                 |132                     |3                     |0 

In [24]:
session.write_pandas(inference_pandas_data,table_name="HR_INFERENCE_DATA",overwrite=True).show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |7.0           |0.78                  |0.99               |4                 |255                     |6                     |0                |1       |0                        |
|1.0       |7.0           |0.45                  |0.51               |2                 |160                     |3                     |1                |1       |1                        |
|1.0       |7.0           |0.76              